In [ ]:
import pathlib
from ultralytics import YOLO

model = YOLO("../../datasets/oor/weights/yolov8s_coco_nc5_best.pt")

In [ ]:

img_list = sorted(list(pathlib.Path("../../datasets/oor/first-training-dataset-oor-nc-5/images/val").glob("*.png")))
batch_paths = img_list[202:204]

In [ ]:
results = model.track(
    source=batch_paths,
    project="../../datasets/oor/first-training-dataset-oor-nc-5",
    imgsz=1024,
    batch=16,
    conf=0.434,
    iou=0.7,
    save_txt=False,
    save_conf=True, 
)

In [ ]:
import numpy as np
import cv2
import os
import pathlib

from ultralytics.engine.results import Boxes

from objectherkenning_openbare_ruimte.data_minimisation import blurring_tools

PERSON = 0
LICENSE_PLATE = 1
CONTAINER = 2

target_classes = (CONTAINER)
sensitive_classes = (PERSON, LICENSE_PLATE)

destination_folder = "../../datasets/oor/test_output"

pathlib.Path(destination_folder).mkdir(parents=True, exist_ok=True)


def _get_annotion_string_from_boxes(boxes: Boxes) -> str:
    boxes = boxes.cpu()

    annotation_lines = []

    for box in boxes:
        cls = int(box.cls.squeeze())
        conf = float(box.conf.squeeze())
        tracking_id = int(box.id.squeeze()) if box.is_track else -1
        yolo_box_str = " ".join([f"{x:.6f}" for x in box.xywhn.squeeze()])
        annotation_lines.append(f"{cls} {yolo_box_str} {conf:.6f} {tracking_id}")
    return "\n".join(annotation_lines)


for r in results:
    result = r.cpu()
    boxes = result.boxes.numpy()

    target_idxs = np.where(np.in1d(boxes.cls, target_classes))[0]
    if len(target_idxs) == 0:  # Nothing to do!
        continue

    image = result.orig_img.copy()
    sensitive_idxs = np.where(np.in1d(boxes.cls, sensitive_classes))[0]

    # Blur sensitive data
    sensitive_bounding_boxes = boxes[sensitive_idxs].xyxy
    image = blurring_tools.blur_inside_boxes(image, sensitive_bounding_boxes)
    
    # Draw annotation boxes
    target_bounding_boxes = boxes[target_idxs].xyxy
    image = blurring_tools.draw_bounding_boxes(image, target_bounding_boxes)

    # Save image
    save_path = os.path.join(destination_folder, pathlib.Path(result.path).name)
    cv2.imwrite(save_path, image)

    # Save annotation
    annotation_str = _get_annotion_string_from_boxes(boxes[target_idxs])
    base_name = pathlib.Path(result.path).stem
    annotation_path = os.path.join(destination_folder, f"{base_name}.txt")
    with open(annotation_path, 'w') as f:
        f.write(annotation_str)